<h3>초기 SETTING</h3>
<ul>
    <li>selenium, beautifulSoup4</li>
    <li>결과 저장 파일 열기</li>
    <li>driver 실행 후 학교 페이지 연결</li> 
</ul>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import re
import json

In [ ]:
jsonFile = open("./rawDataSet.json", "w")
textFile = open("./result.txt", "w") # result.txt에 결과 저장장

# 1. 웹 드라이버 실행 (Chrome 기준)
driver = webdriver.Chrome()

# 2. 특정 URL로 이동
driver.get("https://webinfo.dankook.ac.kr/tiac/univ/lssn/lpci/views/lssnPopup/tmtbl.do")


<h3>교양 시간표 및 강의실 추출</h3>

In [4]:
arts_radios = driver.find_elements(By.NAME, "qrySxn")
arts_radios[0].click()
arts_queryButton = driver.find_element(By.ID, "btn_search")
arts_queryButton.click()

rawDataSet = []

def isSchedule(val: str):
    return '~' in val and '(' in val and ')' in val

def extractInfo(schedule: str):
    day = []
    start = []
    end = [] 
    
    courseRoom = re.search("\([0-9]*[가-힣]*[a-zA-Z]*[0-9]*-*[0-9]*\)", schedule).group()
    courseRoom = courseRoom[1:-1]
    # courseSchedule: list
    courseSchedule = re.findall("[가-힣][0-9]*~[0-9]*", schedule)

    for infoUnitSchedule in courseSchedule:
        day.append(re.match('[가-힣]', infoUnitSchedule).group())
        infoUnitSchedule = infoUnitSchedule[1:]
        period = infoUnitSchedule.split("~")
        start.append(period[0])
        end.append(period[1])
        
    return courseRoom, day, start, end
    
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")   
table = soup.find("table", id = "mjLctTmtblDscTbl")
tbody = table.find("tbody")
setOfRows = tbody.find_all("tr")

i = 1

for row in setOfRows:
    courseName = row.select_one("td:nth-of-type(4)").contents[0].strip()
    professor = row.select_one("td:nth-of-type(8)").contents[0].strip()
    courseMeta = row.select_one("td:nth-of-type(9)").text.strip()

    if not courseMeta or not isSchedule(courseMeta):
        continue  
    
    courseRoom, day, start, end = extractInfo(courseMeta)
    
    rawDataSet.append({
        "courseName": courseName, 
        "professor": professor, 
        "courseRoom": courseRoom, 
        "day": day, 
        "start": start,
        "end": end
    })
    
    i += 1
    
    textFile.write(professor + " " + courseName + " " + courseMeta + "\n")

In [7]:
textFile.close()
jsonFile.close()

<h3>전공 시간표 및 강의실 추출</h3>

In [5]:
major_radios = driver.find_elements(By.NAME, "qrySxn")
major_radios[1].click()
major_queryButton = driver.find_element(By.ID, "btn_search")
major_queryButton.click()

html = driver.page_source

soup = BeautifulSoup(html, "html.parser")   
table = soup.find("table", id = "mjLctTmtblDscTbl")
tbody = table.find("tbody")
setOfRows = tbody.find_all("tr")
for row in setOfRows:
    courseName = row.select_one("td:nth-of-type(4)").contents[0].strip()
    professor = row.select_one("td:nth-of-type(8)").contents[0].strip()
    courseMeta = row.select_one("td:nth-of-type(9)").text.strip()
    
    if not courseMeta or not isSchedule(courseMeta):
        continue  
    
    courseRoom, day, start, end = extractInfo(courseMeta)
    
    rawDataSet.append({
        "courseName": courseName, 
        "professor": professor, 
        "courseRoom": courseRoom, 
        "day": day, 
        "start": start,
        "end": end
    })
    
    i += 1
    
    textFile.write(professor + " " + courseName + " " + courseMeta + "\n")
        
jsonFile.write(json.dumps(rawDataSet, ensure_ascii=False, indent=4))

jsonFile.close()
textFile.close()

print(i)

4486


<h3>MYSQL에 데이터 INSERT</h3>

In [ ]:
import pymysql;

# conn = pymysql.connect(host = "127.0.0.1", user = "SWSSH", password = "1234", db = "DKU_SCHEDULE", charset = "utf8")
conn = pymysql.connect(user='psmsq4',
                       password='root1234!',
                       database='oss_project',
                       host='oss-project.mysql.database.azure.com',
                       ssl={'ca': './DigiCertGlobalRootCA.crt.pem'})
cur = conn.cursor()

f = open("./rawDataSet.json", "r")
jsonFile = json.load(f)

for elem in jsonFile:
    print(elem)

    for i in range(len(elem['day'])):
        day = elem['day'][i]
        start = elem['start'][i]
        end = elem['end'][i]
        try:
            ret = cur.execute(f"INSERT INTO class_room_schedule (CLASS_NAME, CLASS_ROOM, DAY, START, END, PROFESSOR) VALUES('{elem['courseName']}', '{elem['courseRoom']}', '{day}', {start}, {end}, '{elem['professor']}');")
            cur.execute("COMMIT;")
        except Exception as e:
            print(e)

cur.close()    
conn.close()
f.close()

{'courseName': 'SW융합코딩1', 'professor': '김대원', 'courseRoom': '2공521', 'day': ['화'], 'start': ['1'], 'end': ['6']}
{'courseName': 'SW융합코딩1', 'professor': '선경희', 'courseRoom': '소프트301', 'day': ['금'], 'start': ['7'], 'end': ['12']}
{'courseName': 'SW융합코딩1', 'professor': '조남수', 'courseRoom': '소프트416', 'day': ['금'], 'start': ['9'], 'end': ['14']}
{'courseName': 'SW융합코딩1', 'professor': '윤택영', 'courseRoom': '소프트416', 'day': ['금'], 'start': ['1'], 'end': ['6']}
{'courseName': 'SW융합코딩1', 'professor': '송인식', 'courseRoom': '소프트516', 'day': ['목'], 'start': ['1'], 'end': ['6']}
{'courseName': 'SW융합코딩1', 'professor': '서성현', 'courseRoom': '미디어415', 'day': ['수'], 'start': ['11'], 'end': ['16']}
{'courseName': 'SW융합코딩1', 'professor': '김선오', 'courseRoom': '3공519', 'day': ['목'], 'start': ['4'], 'end': ['9']}
{'courseName': '수학일반1', 'professor': '전경배', 'courseRoom': '소프트307', 'day': ['화'], 'start': ['1'], 'end': ['6']}
{'courseName': '수학일반1', 'professor': '이경복', 'courseRoom': '소프트307', 'day': ['수'], 'start

In [8]:
import os
import pyodbc, struct
from azure import identity

from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel

connection_string = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:oss-dankook.database.windows.net,1433;Database=dku_schedule;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

def get_conn():
    credential = identity.DefaultAzureCredential(exclude_interactive_browser_credential=False)
    token_bytes = credential.get_token("https://database.windows.net/.default").token.encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256  # This connection option is defined by microsoft in msodbcsql.h
    conn = pyodbc.connect(connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})
    return conn

def root():
    print("Root of Person API")
    try:
        conn = get_conn()
        cursor = conn.cursor()

        # Table should be created ahead of time in production app.
        cursor.execute("""
            CREATE TABLE CLASS_ROOM_SCHEDULE (
                CLASS_NAME VARCHAR(50) NOT NULL,
                CLASS_ROOM VARCHAR(20) NOT NULL,
                PROFESSOR VARCHAR(10), 
                DAY VARCHAR(4) NOT NULL,
                STARTTIME TINYINT NOT NULL,
                ENDTIME TINYINT NOT NULL,
                PRIMARY KEY (CLASS_ROOM, DAY, STARTTIME)
            );
        """)

        conn.commit()
    except Exception as e:
        # Table may already exist
        print(e)
    return "CLASS_ROOM_SCHEDULE API"

root()


Root of Person API
('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'CLASS_ROOM_SCHEDULE' in the database. (2714) (SQLExecDirectW)")


'CLASS_ROOM_SCHEDULE API'

In [ ]:
def create_person():
    with get_conn() as conn:
        cursor = conn.cursor()
        
        for elem in jsonFile:
            print(elem)

            for i in range(len(elem['day'])):
                day = elem['day'][i]
                start = elem['start'][i]
                end = elem['end'][i]
                try:
                    cursor.execute("INSERT INTO CLASS_ROOM_SCHEDULE (CLASS_NAME, CLASS_ROOM, PROFESSOR, DAY, STARTTIME, ENDTIME) VALUES (?, ?, ?, ?, ?, ?)", (elem['courseName'], elem['courseRoom'], elem['professor'], day, start, end))
                    # ret = cur.execute(f"INSERT INTO COURSEROOM_SCHEDULE (COURSE_NAME, COURSE_ROOM, DAY, START, END, PROFESSOR) VALUES('{elem['courseName']}', '{elem['courseRoom']}', '{day}', {start}, {end}, '{elem['professor']}');")
                    # cur.execute("COMMIT;")
                    conn.commit()
                except Exception as e:
                    print(e)

create_person()

{'courseName': 'SW융합코딩1', 'professor': '김대원', 'courseRoom': '2공521', 'day': ['화'], 'start': ['1'], 'end': ['6']}
('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'COURSEROOM_SCHEDULE'. (208) (SQLExecDirectW)")
{'courseName': 'SW융합코딩1', 'professor': '선경희', 'courseRoom': '소프트301', 'day': ['금'], 'start': ['7'], 'end': ['12']}
('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'COURSEROOM_SCHEDULE'. (208) (SQLExecDirectW)")
{'courseName': 'SW융합코딩1', 'professor': '조남수', 'courseRoom': '소프트416', 'day': ['금'], 'start': ['9'], 'end': ['14']}
('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'COURSEROOM_SCHEDULE'. (208) (SQLExecDirectW)")
{'courseName': 'SW융합코딩1', 'professor': '윤택영', 'courseRoom': '소프트416', 'day': ['금'], 'start': ['1'], 'end': ['6']}
('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'COURSEROOM_SCHEDULE'. (208

OperationalError: ('08S01', '[08S01] [Microsoft][ODBC Driver 18 for SQL Server]Communication link failure (-2147467259) (SQLEndTran(SQL_COMMIT))')